In [1]:
import os
import sys
import json
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models


import models

%load_ext autoreload
%autoreload 2

In [19]:
data_path = "../data/"
training_data_path = 'Data/DataSet13_20200221/raw_patient_based/Training/'
testing_data_path = 'Data/DataSet13_20200221/raw_patient_based/Testing/'
paths = [training_data_path, testing_data_path]
categories = {"Bleeding":1, "NonBleeding":0}

In [102]:
# Training Data
train_data = {}
index_count = 0
index = 0

path = training_data_path
for category in categories:

    current_directory = os.path.join(data_path, path, category)
    print('\t',current_directory)
    for patient_folder in os.listdir(current_directory):
        if not patient_folder.startswith('Patient'):
            continue
        
        patient_path = os.path.join(current_directory, patient_folder)
        
        patient_files = os.listdir(patient_path)
        patient_label = categories[category]
        
        
        elem = {"seq_label": patient_label, "sequence": {}}
        
        for file in patient_files:

            if file.endswith('.tif') or file.endswith('.jpg'):
                if category == 'Bleeding':
                    patient_seq = file.split('.')[0]
                    seq_num = int(patient_seq.split('_')[1])
                    patient_num = patient_seq.split('_')[0]
                    elem["patient_id"] = patient_num
                else:
                    patient_num = file.split('.')[0]
                    seq_num = int(file.split('.')[1])
                    elem["patient_id"] = patient_num
                image_path = os.path.join('/',category, patient_folder, file)
                elem["sequence"][seq_num] = image_path
                
        elem["sequence"] = dict(sorted(elem["sequence"].items(), key=lambda t: t[0]))
        
        train_data[index_count] = elem
        index_count += 1


	 ../data/Data/DataSet13_20200221/raw_patient_based/Training/Bleeding
	 ../data/Data/DataSet13_20200221/raw_patient_based/Training/NonBleeding


In [103]:
f = open("train_meta.json", 'w') 
json.dump(train_data, f)

In [106]:
# Testing Data
test_data = {}
index_count = 0
index = 0

path = testing_data_path
for category in categories:

    current_directory = os.path.join(data_path, path, category)
    print('\t',current_directory)
    for patient_folder in os.listdir(current_directory):
        if not patient_folder.startswith('Patient'):
            continue
        
        patient_path = os.path.join(current_directory, patient_folder)
        
        patient_files = os.listdir(patient_path)
        patient_label = categories[category]
        
        
        elem = {"seq_label": patient_label, "sequence": {}}
        
        for file in patient_files:
                if file.endswith('.jpg'):
                    patient_seq = file.split('.')[0]
                    seq_num = int(patient_seq.split('_')[1])
                    patient_num = patient_seq.split('_')[0]
                    elem["patient_id"] = patient_num
                    image_path = os.path.join('/',category, patient_folder, file)
                    elem["sequence"][seq_num] = image_path

                image_path = os.path.join('/',category, patient_folder, file)
                elem["sequence"][seq_num] = image_path
                
        elem["sequence"] = dict(sorted(elem["sequence"].items(), key=lambda t: t[0]))
        
        test_data[index_count] = elem
        index_count += 1


	 ../data/Data/DataSet13_20200221/raw_patient_based/Testing/Bleeding
	 ../data/Data/DataSet13_20200221/raw_patient_based/Testing/NonBleeding


In [108]:
f = open("test_meta.json", 'w') 
json.dump(test_data, f)

# Demonstration of how to use data

In [109]:
meta_data = json.load(open('train_meta.json'))

In [110]:
patient_ids = list(meta_data.keys())

In [111]:
patient_data = patient_ids[0]

In [112]:
meta_data[patient_data]

{'seq_label': 1,
 'sequence': {'7': '/Bleeding/Patient 54/54_7.tif',
  '35': '/Bleeding/Patient 54/54_35.tif',
  '37': '/Bleeding/Patient 54/54_37.tif',
  '41': '/Bleeding/Patient 54/54_41.tif',
  '44': '/Bleeding/Patient 54/54_44.tif',
  '48': '/Bleeding/Patient 54/54_48.tif',
  '52': '/Bleeding/Patient 54/54_52.tif',
  '54': '/Bleeding/Patient 54/54_54.tif'},
 'patient_id': '54'}

# Demonstration of using the Models

In [4]:
from dataloader import trainloader
from torchvision import transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406),
                         (0.229, 0.224, 0.225))])

In [5]:
dl = trainloader(transform=transform)

In [20]:
img, labels = dl[0]
img = img.unsqueeze(0)

In [43]:
from models import EncoderCNN, DecoderRNN

In [44]:
encoder = EncoderCNN()

In [45]:
cnn_op = encoder(img)

In [35]:
cnn_op.shape

torch.Size([11, 1, 4096])

In [46]:
decoder = DecoderRNN()

In [47]:
lstm_op = decoder(cnn_op)

/home/deepandas11/anaconda3/envs/ml/lib/python3.7/site-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [48]:
lstm_op

tensor([[0.4999]], grad_fn=<SigmoidBackward>)

In [58]:
1 + 1 == 3 and 2 / 0

False